In [1]:
import numpy as np
import os
import csv
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import warnings
import dash
from dash import Dash, dcc, html, Input, Output, callback, dash_table, State, MATCH, ALL, Patch, State, ctx, register_page, page_container, no_update
import dash_bootstrap_components as dbc
import time
from plotly.subplots import make_subplots
from scipy.optimize import minimize
import pymysql
import dash_daq as daq
import datetime
import time

In [2]:
app = dash. Dash(__name__,  title='Робоадвайзинг - портфель', suppress_callback_exceptions=True)

In [3]:
STYLE_MAIN_BLOCK = {
    "position": "absolute", 
    "top": '1rem', # Небольшой отступ сверху
    "left": '10%', 
    "right": '10%',
    "width": "auto", # Автоматическая ширина, чтобы блок занимал остаток пространства
    "padding": "1rem 2rem", 
    "background-color": "#F3F3F2",
    "border-radius": "20px", # Закругленные углы
}
colors_pie = ['rgb(193,160,104)', 'rgb(216,198,164)', 'rgb(236,227,210)', 'rgb(61,81,90)', 'rgb(193,160,104)', 'rgb(216,198,164)', 'rgb(236,227,210)', 'rgb(61,81,90)']
options_region  = {
                                'Европа': ['Только для себя', 'Для всей семьи'],
                                'Центральная Америка': [],
                                'Азия': [],
                                'Южная Америка': [],
                                'Океания': [],
                                'Северная Америка': [],
                                'Африка': [],
                                'Не важно': [],
                            }

In [4]:
def read_data():
    global USER_DATA, USER_DATA_TABLE
    USER_DATA = pd.read_csv('/Users/erickgaydin/Desktop/USER_DATA.csv')
    USER_DATA_TABLE = pd.read_csv('/Users/erickgaydin/Desktop/USER_DATA_TABLE.csv')

## Функция, которая делает оптимизацию портфеля

In [5]:
def make_per_stonks(total_amount, main_data):
    global fig
    prices = list(main_data['Сумма Единовременный взнос'])  # Цены товаров
    names = list(main_data['NAME'])  # Названия товаров
    rates = list(main_data['Yield_percent'])  # Ставки товаров
    
    items = list(zip(prices, names, rates))
    
    # Сортируем товары по ставке в убывающем порядке
    items.sort(key=lambda x: x[2], reverse=True)
    
    total_cost_of_all_items = sum(prices)
    affordable_items = {}
    spent_amount = 0
    
    if total_amount >= total_cost_of_all_items:
        # Если можем купить все товары
        total_quantity = total_amount / total_cost_of_all_items
        for price, name, rate in items:
            quantity = total_quantity * (price / price)
            affordable_items[name] = quantity
            spent_amount += quantity * price
    else:
        # Если не можем купить все товары
        remaining_amount = total_amount
        
        # Сначала пытаемся купить товары по одному, если это возможно
        for price, name, rate in items:
            if price <= remaining_amount:
                affordable_items[name] = 1
                spent_amount += price
                remaining_amount -= price
        
        # Перераспределяем оставшиеся средства
        while remaining_amount > 0:
            best_item = None
            best_value = 0
            
            for price, name, rate in items:
                if name in affordable_items:
                    # Если товар уже в корзине, пытаемся увеличить его количество
                    if price <= remaining_amount:
                        quantity = remaining_amount / price
                        if quantity > 0 and rate > best_value:
                            best_item = (name, price, quantity)
                            best_value = rate
                else:
                    # Если товар не в корзине, проверяем возможность покупки
                    if price <= remaining_amount:
                        quantity = remaining_amount / price
                        if quantity > 0 and rate > best_value:
                            best_item = (name, price, quantity)
                            best_value = rate
            
            if not best_item:
                break

            name, price, quantity = best_item
            affordable_items[name] = affordable_items.get(name, 0) + quantity
            spent_amount += quantity * price
            remaining_amount -= quantity * price

    # Рассчитываем процент потраченной суммы
    percentage_spent = (spent_amount / total_amount) * 100

    # Создаем таблицу 
    data = {
        'Название': [],
        'Количество': [],
        'Стоимость': []
    }
    
    for name, quantity in affordable_items.items():
        price = prices[names.index(name)]
        data['Название'].append(name)
        data['Количество'].append(quantity)
        data['Стоимость'].append(quantity * price)
    
    df = pd.DataFrame(data)
    
    fig = px.pie(df, values='Стоимость', names='Название', hole=0.7, color_discrete_sequence = colors_pie[:len(main_data)])
    fig.update_traces(textposition='inside', textinfo='label')
    

In [6]:
def make_income_bar_by_year(USER_DATA):
    
    global fig_inc
    
    years = int(USER_DATA['investyear'][0])
    sum = int(USER_DATA['suminvest'][0])
    type = int(USER_DATA['portfeltype'][0])
    if type==1:
        i_i = 0.09
        FV_years = []
        FV_years_min5 = []
        FV_years_plus5 = []
        year_sum = []
        for t in range(years+1):
            FV = sum * (1 + i_i)**(t)
            FV_years.append(FV)
            FV_years_min5.append(FV*0.95)
            FV_years_plus5.append(FV*1.05)
            year_sum.append(2024 + t)
    elif type==2:
        i_i = 0.06
        FV_years = []
        FV_years_min5 = []
        FV_years_plus5 = []
        year_sum = []
        for t in range(years+1):
            FV = sum * (1 + i_i)**(t)
            FV_years.append(FV)
            FV_years_min5.append(FV*0.95)
            FV_years_plus5.append(FV*1.05)
            year_sum.append(2024 + t)
        
    elif type==3:
        i_i = 0.14
        FV_years = []
        FV_years_min5 = []
        FV_years_plus5 = []
        year_sum = []
        for t in range(years+1):
            FV = sum * (1 + i_i)**(t)
            FV_years.append(FV)
            FV_years_min5.append(FV*0.95)
            FV_years_plus5.append(FV*1.05)
            year_sum.append(2024 + t)

    fig_inc = go.Figure()
    fig_inc.add_trace(go.Bar(
                            x=year_sum,
                            y=FV_years,
                            name='Базовый сценарий',
                            marker_color = 'rgb(216,198,164)', marker_line=dict(color='rgba(0,0,0,0)', width=1)
                        ))
    fig_inc.add_trace(go.Scatter(x=year_sum, y=FV_years_min5, name="Негативный сценарий",
                    line_shape='spline'))
    fig_inc.add_trace(go.Scatter(x=year_sum, y=FV_years_plus5, name="Позитивный сценарий",
                    line_shape='spline'))
    fig_inc.update_layout(
                            # Убираем фон
                            plot_bgcolor='rgba(0, 0, 0, 0)',
                            
                            # Заголовок по центру
                            title={
                                'text': "Планируемая доходность",
                                'x': 0.5,  # Позиция по оси X (0.5 — это центр)
                                'xanchor': 'center',
                                'yanchor': 'top'
                            },

                            bargap=0.5,
                        
                            # Убираем легенду (опционально)
                            showlegend=True
                        )    
    

## Рендер портфеля

In [7]:
def render_portret():
    
    global main_data, USER_RETURN
    """Читаем данные клиента"""
    read_data()
    """Читаем данные услуг"""
    main_data = pd.read_excel('/Users/erickgaydin/Desktop/Work.Finion/FG/table_opros.xlsx')
    """Определяем, какой тип портфеля выбрал клиент"""
    invest_income = 10
    if USER_DATA['portfeltype'][0]  == 1: #Умеренный
        option_suit_1 = "normal"
        invest_income = 9
    elif USER_DATA['portfeltype'][0]  == 2: #Консервативный
        option_suit_1 = "minimal_risk"
        invest_income = 6
    else: # Агресивный
        option_suit_1 = "agressive" 
        invest_income = 14

    """Сортируем таблицу по данным клиента"""
    main_data = main_data[(main_data['Horizont_year'] <= USER_DATA['investyear'][0] ) & (main_data[option_suit_1] == 1) & (main_data['Сумма Единовременный взнос']<=USER_DATA['suminvest'][0])]
    main_data = main_data
    
    if invest_income < main_data['Yield_percent'].min() :
        USER_RETURN = html.H5(id="22222", children = " Увы и Ах! Но Вам либо увеличить кол-во лет инвестирования, либо стратегию портфеля",
                        style = {
                                 'textAlign': 'center',  'color': '#333332'
                                }
                        )
    else:     
        make_per_stonks(USER_DATA['suminvest'][0], main_data)
        make_income_bar_by_year(USER_DATA)
        USER_RETURN =  html.Div([
                               
                              html.Div([
                                          dash_table.DataTable(   
                                                                    id='table',
                                                                    columns=[{"name": i, "id": i} for i in main_data[["NAME","Продукт","Jurisdiction","Сумма Единовременный взнос"]].columns],
                                                                    style_data={
                                                                                'whiteSpace': 'normal',
                                                                                'height': 'auto',
                                                                                'color': 'black',
                                                                                'backgroundColor': 'white',
                                                                                'border': 'none'  # Убираем все границы для данных
                                                                                },
                                                                    style_table={'overflowX': 'auto'},
                                                                    style_header={
                                                                                    'backgroundColor': '#C1A068',
                                                                                    'color': 'white',
                                                                                    'fontWeight': 'bold',
                                                                                    'borderBottom': '1px solid black'  # Линия только между заголовками колонок и данными
                                                                                    },
                                                                    data=main_data[["NAME","Продукт","Jurisdiction","Сумма Единовременный взнос"]].to_dict('records')
                                                                )
                                      ]),
                                html.Hr(),
                                html.H1(id = 'TEXT_1_1', children = 'Сводная информация',
                                                style = {
                                                         'textAlign': 'center',  'color': '#333332', 'font-weight': '2'
                                                        }
                                                ), 
                                html.Div([ 
                                            dcc.Graph(figure=fig, style={'width': '100%', 'height': '100%'}) 
                                         ], style   = {

                                                            "top": '1rem', 
                                                            "left": '50%', # Начало блока от середины страницы
                                                            "height": "calc(100% - 2rem)", # Высота блока с отступом от нижнего края
                                                            "width": "70%", # Ширина блока с отступом от правого края
                                                            "padding": "1rem 2rem", 
                                                            "background-color": "#FFFEFE",
                                                            "border-radius": "20px", # Закругленные углы
                                                        }),
                                 html.Hr(),
                                 html.Div([ 
                                             dcc.Graph(figure=fig_inc)
                                         ], style   = {"border-radius": "20px", "padding": "1rem 2rem", "background-color": "#FFFEFE"})
           
                                                                            
                        ])
    

In [8]:
@app.callback(
    Output('outpu_data_10', 'children'),
    Input('interval-component', 'n_intervals')
)
def update_data(n):
    render_portret()
    return USER_RETURN

## Гражданства 

In [9]:
@callback(  Output('content_passport_table', 'children',allow_duplicate=False),
            Input('region_of_passport', 'value'),
            prevent_initial_call=True)
def render_portret_passport(region_of_passport ): 
    global df
    df = pd.read_csv('/Users/erickgaydin/Desktop/Work.Finion/FG/PassportGroups-Grid view.csv')
    df = df[["Country",  "Region_lookup", "Schengen_lookup", "VisaFree", "Status", 'Client_description']]
    df['id'] = df['Country'].astype(str)
    df.set_index('id', inplace=True, drop=False)
    if region_of_passport == "Не важно":
        df = df[df['Status'] == "Доступно"]
        df['Client_description'] = df['Client_description'].str.slice(25,170)
    else:
        df = df[(df['Region_lookup']==region_of_passport) & (df['Status'] == "Доступно")]
        df['Client_description'] = df['Client_description'].str.slice(25,170)
    return html.Div([
                                  dash_table.DataTable(   
                                                            id='table_0912348',
                                                            columns=[{"name": i, "id": i} for i in df.columns if i != 'id'],
                                                                    style_data={
                                                                                'whiteSpace': 'normal',
                                                                                'height': 'auto',
                                                                                'color': 'black',
                                                                                'backgroundColor': 'white',
                                                                                'border': 'none'  # Убираем все границы для данных
                                                                                },
                                                                    style_table={'overflowX': 'auto'},
                                                                    style_header={
                                                                                    'backgroundColor': '#C1A068',
                                                                                    'color': 'white',
                                                                                    'fontWeight': 'bold',
                                                                                    'borderBottom': '1px solid black'  # Линия только между заголовками колонок и данными
                                                                                    },
                                                            data=df.to_dict('records'),
                                                            row_selectable="multi",
                                                            row_deletable=False,
                                                            selected_rows=[],
                                                            page_action="native",  page_size= 10
                                                        ),
                            html.Br()           
                            ])

## Страхование

In [10]:
@app.callback(
    Output('outpu_data_11', 'children'),
    Input('interval-component', 'n_intervals')
)
def update_data(n):
    render_insurance()
    return DIV_INSURANCE

In [11]:
def render_insurance():  
    
    global data_insurance, dict_insurance, DIV_INSURANCE

    data_insurance = pd.read_excel('/Users/erickgaydin/Desktop/Work.Finion/FG/страхование.xlsx')
    data_insurance = data_insurance[data_insurance['Стоимость в год']<=USER_DATA['sumstrah'][0]]
    TEXT_4294923 = f"Ваша сумма для инвестирования в год {USER_DATA['sumstrah'][0]} $"
    dict_insurance = data_insurance['Продукт'].to_dict()
    DIV_INSURANCE = html.Div([ 
                        html.H5(id="4294923", children = TEXT_4294923,
                                style = {
                                         'textAlign': 'left',  'color': '#333332'
                                        }
                               ),
                        dash_table.DataTable(   
                                                id='table_insurance',
                                                columns=[{"name": i, "id": i} for i in data_insurance[["Продукт","Юрисдикция", "Стоимость в год", "Примечание"]].columns],
                                                                    style_data={
                                                                                'whiteSpace': 'normal',
                                                                                'height': 'auto',
                                                                                'color': 'black',
                                                                                'backgroundColor': 'white',
                                                                                'border': 'none'  # Убираем все границы для данных
                                                                                },
                                                                    style_table={'overflowX': 'auto'},
                                                                    style_header={
                                                                                    'backgroundColor': '#C1A068',
                                                                                    'color': 'white',
                                                                                    'fontWeight': 'bold',
                                                                                    'borderBottom': '1px solid black'  # Линия только между заголовками колонок и данными
                                                                                    },
                                                data=data_insurance[["Продукт","Юрисдикция", "Стоимость в год", "Примечание"]].to_dict('records')
                                             ),
                        html.Br(),
                        html.Div([ 
                                    dcc.RadioItems(id='wich_insurance',
                                                  style={'marginRight':'10px',
                                                     'width': 1200,
                                                     'margin': '0 auto',
                                                     'display': 'flex',
                                                     'justify-content': 'center', 'color':'#333332', 'flex-wrap': 'wrap', 'gap': '15px',
                                                     })
                                ], style = {
                                            "left": '10%', # Отступ слева на 5%
                                            "right": '30%', # Отступ справа на 5%
                                            'display': 'flex', 'justify-content': 'center'
                                            })
                    ])

In [12]:
@callback(
    Output('wich_insurance', 'options'),
    Input('region_of_passport', 'value'))
def set_cities_options(tab_insurance):
    dict_insurance = data_insurance['Продукт'].to_dict()
    return [{'label': i, 'value': i} for i in dict_insurance.values()]

## Рендер таблицы целей

In [13]:
def render_table_goals():
    global DIV_TABLE
    """Читаем данные клиента"""
    read_data()
    """Записываем основные константы"""
    invest_age = USER_DATA['investyear'][0]
    invest_sum = USER_DATA['suminvest'][0]
    client_year = USER_DATA['age'][0]
    monthly_pay = invest_sum * ((1 + 10 /100) ** invest_age) / invest_age
    """Преобразуем таблицу"""
    """Делаем копию исходной таблицы"""
    data_main = USER_DATA_TABLE.copy()

    # 2. Изменение колонок с помощью .loc[]
    data_main['Кол-во оставшихся лет до достижения'] = data_main['Возраст достижения цели'].astype(int) - int(client_year)
    data_main = data_main[['Название цели','Стоимость цели, в $','Кол-во оставшихся лет до достижения']]
    data_main['Ежегод. взнос, в $'] = data_main['Стоимость цели, в $'].astype(int) / data_main['Кол-во оставшихся лет до достижения'].astype(int)
    data_main['Возможно ли'] = np.where(data_main['Ежегод. взнос, в $']<monthly_pay, 'yes', 'no')
    data_main['Сколько не хватает'] = data_main['Ежегод. взнос, в $'] - monthly_pay
    data_main['Сколько не хватает'] = np.where(data_main['Сколько не хватает']>0, data_main['Сколько не хватает'], 0)
    data_main = data_main.round(2)
    """Переменная для вывода html-компоненты"""
    DIV_TABLE = html.Div([
                            html.H3(id="22", children = 'Возможность реализации целей',
                                            style = {
                                                     'textAlign': 'center',  'color': '#333332'
                                                    }
                                            ),
                            dash_table.DataTable(   
                                                    id='table_insurance',
                                                    columns=[{"name": i, "id": i} for i in data_main.columns],
                                                    style_data={
                                                                'whiteSpace': 'normal',
                                                                'height': 'auto',
                                                                'color': 'black',
                                                                'backgroundColor': 'white',
                                                                'border': 'none'  # Убираем все границы для данных
                                                                },
                                                    style_table={'overflowX': 'auto'},
                                                    style_data_conditional=[
                                                                      {
                                                                            'if': {
                                                                                'column_id': 'Возможно ли',
                                                                                'filter_query': '{Возможно ли} eq "yes"'
                                                                            },
                                                                            'backgroundColor': '#A8E4A0',
                                                                        },
                                                                        {
                                                                            'if': {
                                                                                'column_id': 'Возможно ли',
                                                                                'filter_query': '{Возможно ли} eq "no"'
                                                                            },
                                                                            'backgroundColor': '#F34723',
                                                                        }
                                                                                ],
                                                     style_header={
                                                                    'backgroundColor': '#C1A068',
                                                                    'color': 'white',
                                                                    'fontWeight': 'bold',
                                                                    'borderBottom': '1px solid black'  # Линия только между заголовками колонок и данными
                                                                    },
                                                    data=data_main.to_dict('records')
                                                 ),
        
        
                        ])

In [14]:
@app.callback(
    Output('outpu_data_12', 'children'),
    Input('interval-component', 'n_intervals')
)
def update_data(n):
    render_table_goals()
    return DIV_TABLE

## Скелет приложения

In [15]:
app.layout = html.Div([  dcc.Interval(
                                            id='interval-component',
                                            interval=60000, # Интервал в миллисекундах (60000 = 1 минута)
                                            n_intervals=0,   # Начальное значение счетчика интервалов
                                            disabled=False    # Включено по умолчанию
                                        ),
                         html.Div([
                                        html.H1(id = 'TEXT_1', children = 'Возможности по инвестициям',
                                                style = {
                                                         'textAlign': 'center',  'color': '#333332', 'font-weight': '2'
                                                        }
                                                ),
                                         render_portret(),
                                         html.Div(id = "outpu_data_10"),
                                         html.H1(id = 'TEXT_123', children = 'Возможности по страхованию',
                                                style = {
                                                         'textAlign': 'center',  'color': '#333332', 'font-weight': '2'
                                                        }
                                                ),
                                         render_insurance(),
                                         html.Div(id = "outpu_data_11"),
                                         html.H1(id = 'TEXT_12343', children = 'Возможности по гражданству',
                                                style = {
                                                         'textAlign': 'center',  'color': '#333332', 'font-weight': '2'
                                                        }
                                                ),
                                         html.Div([ 
                        
                                                    dcc.RadioItems(
                                                                    list(options_region.keys()),
                                                                    'Неизвестный',
                                                                    id='region_of_passport',
                                                                    style={  'marginRight':'10px',
                                                                             'width': 1200,
                                                                             'margin': '0 auto',
                                                                             'display': 'flex',
                                                                             'justify-content': 'center', 'color':  '#333332'
                                                                            }
                                                                )
                                                    ]),
                                         html.Img(src = 'https://www.proslide.com/wp-content/uploads/2021/01/world-map-temp.png',
                                             style={ 
                                                     'width': 900,
                                                     'margin': '0 auto',
                                                     'display': 'flex',
                                                     'justify-content': 'center'
                                                   }
                                            ),
                                         html.Div(id='content_passport_table'),
                                         render_table_goals(),
                                         html.Div(id = 'outpu_data_12')
                                         
                             
                                  ], style = STYLE_MAIN_BLOCK)
    
                      ], style = {"background-color": "#333332"})
if __name__ == '__main__':
    app.run_server(port=1112, host="0.0.0.0")